In [1]:
import os
import cv2
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array

In [ ]:
model_dir = "tflite/deteksi/best_float32.tflite"

interpreter = tf.lite.Interpreter(model_path=model_dir)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

def enlarge_bbox(x1, y1, x2, y2, scale=1.2, img_width=640, img_height=640):
    w = x2 - x1
    h = y2 - y1
    cx = x1 + w / 2
    cy = y1 + h / 2

    new_w = w * scale
    new_h = h * scale

    new_x1 = max(0, int(cx - new_w / 2))
    new_y1 = max(0, int(cy - new_h / 2))
    new_x2 = min(img_width, int(cx + new_w / 2))
    new_y2 = min(img_height, int(cy + new_h / 2))

    return new_x1, new_y1, new_x2, new_y2

c:\Users\yosua\Downloads\klasifikasi-jerawat\venv\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
img_path = "acne_list.jpg"
img = cv2.imread(img_path)
img_height, img_width = img.shape[:2]

input_shape = input_details[0]['shape'][1:3]
input_tensor = cv2.resize(img, tuple(input_shape))
input_tensor = input_tensor.astype(np.float32) / 255.0
input_tensor = np.expand_dims(input_tensor, axis=0)

interpreter.set_tensor(input_details[0]['index'], input_tensor)
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])

detections = output_data[0] 

folder_dir = "hasil_crop"
os.makedirs(folder_dir, exist_ok=True)

for i, det in enumerate(detections):
    x1, y1, x2, y2, conf, cls = det[:6]
    if conf < 0.65:
        continue

    scale_x = img_width / input_shape[1]
    scale_y = img_height / input_shape[0]

    x1 = int(x1 * scale_x)
    y1 = int(y1 * scale_y)
    x2 = int(x2 * scale_x)
    y2 = int(y2 * scale_y)

    x1, y1, x2, y2 = enlarge_bbox(x1, y1, x2, y2, scale=1.75, img_width=img_width, img_height=img_height)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    crop = img[y1:y2, x1:x2]
    scale_factor = 4.5
    resized_crop = cv2.resize(crop, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
    crop_path = f"{folder_dir}/crop_{i}.jpg"
    cv2.imwrite(crop_path, resized_crop)

In [4]:
model_path = "tflite/klasifikasi/model.tflite"
interpreter_clf = tf.lite.Interpreter(model_path=model_path)
interpreter_clf.allocate_tensors()

input_details_clf = interpreter_clf.get_input_details()
output_details_clf = interpreter_clf.get_output_details()

with open("tflite/klasifikasi/labels.json", "r") as f:
    class_indices = json.load(f)

class_names = {v: k for k, v in class_indices.items()}

In [ ]:
def predict_image(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            img_path = os.path.join(folder_path, filename)

            img = load_img(img_path, target_size=(224, 224))
            img_array = img_to_array(img).astype(np.float32)
            img_array = img_array / 255.0
            img_array = np.expand_dims(img_array, axis=0)

            interpreter_clf.set_tensor(input_details_clf[0]['index'], img_array)
            interpreter_clf.invoke()
            output_data = interpreter_clf.get_tensor(output_details_clf[0]['index'])

            pred_index = np.argmax(output_data)
            confidence = float(output_data[0][pred_index]) * 100.0
            class_label = class_names[pred_index]

            plt.figure()
            plt.imshow(load_img(img_path))
            plt.axis('off')
            plt.title(f"{filename} : {class_label} ({confidence:.2f}%)")
            plt.show()

In [6]:
predict_image("hasil_crop")